# Heart Disease Prediction System Tutorial

This tutorial demonstrates how to use the Heart Disease Prediction system, from data preprocessing to model training and making predictions using the API.

## Table of Contents

1. [Setup and Installation](#setup)
2. [Data Exploration](#data-exploration)
3. [Feature Engineering](#feature-engineering)
4. [Model Training](#model-training)
5. [Model Evaluation](#model-evaluation)
6. [Using the API](#using-the-api)
7. [Batch Processing](#batch-processing)
8. [Caching for Performance](#caching)
9. [System Architecture](#system-architecture)
10. [Next Steps](#next-steps)

## Setup and Installation <a id="setup"></a>

First, let's set up our environment and install the necessary packages.

In [ ]:
# Install necessary packages
%pip install -q pandas numpy scikit-learn tensorflow matplotlib seaborn requests

In [ ]:
# Import libraries
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    precision_recall_curve,
)
from sklearn.model_selection import train_test_split

# Set up notebook display
%matplotlib inline
plt.style.use("ggplot")
sns.set(style="whitegrid")
pd.set_option("display.max_columns", None)

### Project Structure

Let's examine the project structure to understand how the system is organized.

In [ ]:
# Define the project root
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"Project root: {PROJECT_ROOT}")

# List key directories
for dir_name in ["data", "models", "src", "api", "config", "docs", "reports"]:
    path = os.path.join(PROJECT_ROOT, dir_name)
    if os.path.exists(path):
        print(f"\n{dir_name}/ directory contents:")
        print("\n".join(f"  - {f}" for f in os.listdir(path) if not f.startswith(".")))

## Data Exploration <a id="data-exploration"></a>

Let's explore the heart disease dataset to understand its features and characteristics.

In [ ]:
# Add the project root to the Python path
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# Load processed data
try:
    processed_data_path = os.path.join(
        PROJECT_ROOT, "data/processed/processed_data.npz"
    )
    data = np.load(processed_data_path)
    X = data["X"]
    y = data["y"]
    feature_names = data["feature_names"]

    # Convert to DataFrame for easier exploration
    df = pd.DataFrame(X, columns=feature_names)
    df["target"] = y
    print(
        f"Loaded processed data with {df.shape[0]} samples and {df.shape[1]} features (including target)"
    )
except FileNotFoundError:
    print("Processed data not found. Let's load the raw data instead.")
    raw_data_path = os.path.join(PROJECT_ROOT, "data/raw/heart_disease_combined.csv")
    df = pd.read_csv(raw_data_path)
    print(f"Loaded raw data with {df.shape[0]} samples and {df.shape[1]} columns")

In [ ]:
# Display the first few rows of the data
df.head()

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values per column:")
for col, count in missing_values.items():
    if count > 0:
        print(f"  - {col}: {count} missing values")
if missing_values.sum() == 0:
    print("  No missing values found")

In [ ]:
# Display summary statistics
df.describe()

In [ ]:
# Distribution of the target variable
plt.figure(figsize=(8, 6))
sns.countplot(x="target", data=df)
plt.title("Heart Disease Distribution")
plt.xlabel("Heart Disease (0=No, 1=Yes)")
plt.ylabel("Count")
target_counts = df["target"].value_counts()
for i, count in enumerate(target_counts):
    plt.text(i, count + 10, f"{count} ({count/len(df):.1%})")
plt.show()

In [ ]:
# Explore relationships between numeric features and the target
numeric_features = df.select_dtypes(include=["float64", "int64"]).columns.tolist()
if "target" in numeric_features:
    numeric_features.remove("target")

# Plot histograms for numeric features by target class
fig, axes = plt.subplots(
    nrows=len(numeric_features) // 2 + (len(numeric_features) % 2),
    ncols=2,
    figsize=(16, 4 * len(numeric_features) // 2),
)
axes = axes.flatten()

for i, feature in enumerate(numeric_features):
    sns.histplot(data=df, x=feature, hue="target", kde=True, ax=axes[i])
    axes[i].set_title(f"Distribution of {feature} by Heart Disease")
    axes[i].set_xlabel(feature)

# Hide unused subplots if any
for i in range(len(numeric_features), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# Correlation matrix
plt.figure(figsize=(14, 10))
correlation_matrix = df.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", mask=mask)
plt.title("Feature Correlation Matrix")
plt.tight_layout()
plt.show()

## Feature Engineering <a id="feature-engineering"></a>

Let's explore the feature engineering process used in the project.

In [ ]:
# Import the feature engineering module
from src.features.feature_engineering import (
    create_feature_interactions,
    create_medical_risk_score,
)
from src.utils import load_config

# Load configuration
config = load_config()
print("Loaded configuration:")
print(json.dumps(config.get("preprocessing", {}), indent=2))

In [ ]:
# Demonstrate feature interactions
df_with_interactions = df.copy()
interactions_data = create_feature_interactions(df.drop("target", axis=1))
for col in interactions_data.columns:
    df_with_interactions[col] = interactions_data[col]

print(f"Original dataset: {df.shape[1]} features")
print(f"With interactions: {df_with_interactions.shape[1]} features")
print("\nNew interaction features:")
interaction_cols = [col for col in df_with_interactions.columns if "_x_" in col]
for col in interaction_cols[:5]:  # Show first 5 interaction features
    print(f"  - {col}")
if len(interaction_cols) > 5:
    print(f"  ... and {len(interaction_cols) - 5} more")

In [ ]:
# Demonstrate medical risk score calculation
risk_score_df = df.copy()
risk_score = create_medical_risk_score(df.drop("target", axis=1))
risk_score_df["medical_risk_score"] = risk_score

# Visualize the medical risk score distribution by heart disease status
plt.figure(figsize=(10, 6))
sns.histplot(
    data=risk_score_df,
    x="medical_risk_score",
    hue="target",
    kde=True,
    element="step",
    bins=20,
)
plt.title("Medical Risk Score Distribution by Heart Disease")
plt.xlabel("Medical Risk Score")
plt.show()

## Model Training <a id="model-training"></a>

Let's explore how models are trained in this project.

In [ ]:
# Import model training functions
from src.models.mlp_model import build_sklearn_mlp, build_keras_mlp

# Load model configurations
print("MLP Model configurations:")
print(json.dumps(config.get("model", {}), indent=2))

In [ ]:
# Split data for demonstration
from sklearn.preprocessing import StandardScaler

X = df.drop("target", axis=1).values  # Use just the raw dataset for simplicity
y = df["target"].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
# Build and train a small scikit-learn MLP for demonstration
sklearn_params = {
    "hidden_layer_sizes": (50, 25),
    "activation": "relu",
    "solver": "adam",
    "alpha": 0.0001,
    "learning_rate_init": 0.001,
    "max_iter": 200,  # Small value for demo
    "random_state": 42,
}

# Build model
sklearn_mlp = build_sklearn_mlp(params=sklearn_params)

# Train model
sklearn_mlp.fit(X_train_scaled, y_train)

# Evaluate
sklearn_score = sklearn_mlp.score(X_test_scaled, y_test)
print(f"scikit-learn MLP Accuracy: {sklearn_score:.4f}")

In [ ]:
# Build and train a small Keras MLP for demonstration
import tensorflow as tf
from tensorflow import keras

# Make it reproducible
tf.random.set_seed(42)

# Build model
input_dim = X_train_scaled.shape[1]
keras_mlp = build_keras_mlp(
    input_dim,
    architecture=[
        {"units": 64, "activation": "relu", "dropout": 0.2, "l2_regularization": 0.01},
        {"units": 32, "activation": "relu", "dropout": 0.2, "l2_regularization": 0.01},
    ],
)

# Compile model
keras_mlp.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

# Early stopping callback
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

# Train model (with a small number of epochs for demonstration)
history = keras_mlp.fit(
    X_train_scaled,
    y_train,
    epochs=30,  # Small value for demo
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping],
    verbose=0,
)

# Evaluate
_, keras_score = keras_mlp.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Keras MLP Accuracy: {keras_score:.4f}")

In [ ]:
# Plot Keras training history
plt.figure(figsize=(12, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Training")
plt.plot(history.history["val_accuracy"], label="Validation")
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Training")
plt.plot(history.history["val_loss"], label="Validation")
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend()

plt.tight_layout()
plt.show()

## Model Evaluation <a id="model-evaluation"></a>

Let's evaluate the model performance more thoroughly.

In [ ]:
# Evaluate scikit-learn model
from src.models.mlp_model import combine_predictions

# Get predictions
sklearn_preds = sklearn_mlp.predict(X_test_scaled)
sklearn_probs = sklearn_mlp.predict_proba(X_test_scaled)[:, 1]

# Get Keras predictions
keras_probs = keras_mlp.predict(X_test_scaled).flatten()
keras_preds = (keras_probs >= 0.5).astype(int)

# Combine predictions
ensemble_probs = combine_predictions(sklearn_probs, keras_probs, method="mean")
ensemble_preds = (ensemble_probs >= 0.5).astype(int)

In [ ]:
# Classification reports
print("scikit-learn MLP Classification Report:")
print(classification_report(y_test, sklearn_preds))

print("\nKeras MLP Classification Report:")
print(classification_report(y_test, keras_preds))

print("\nEnsemble Classification Report:")
print(classification_report(y_test, ensemble_preds))

In [ ]:
# Plot confusion matrices
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title(title)
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()


plot_confusion_matrix(y_test, sklearn_preds, "scikit-learn MLP Confusion Matrix")
plot_confusion_matrix(y_test, keras_preds, "Keras MLP Confusion Matrix")
plot_confusion_matrix(y_test, ensemble_preds, "Ensemble Confusion Matrix")

In [ ]:
# Plot ROC curves
plt.figure(figsize=(10, 8))

# scikit-learn MLP
fpr_sklearn, tpr_sklearn, _ = roc_curve(y_test, sklearn_probs)
roc_auc_sklearn = auc(fpr_sklearn, tpr_sklearn)
plt.plot(
    fpr_sklearn, tpr_sklearn, label=f"scikit-learn MLP (AUC = {roc_auc_sklearn:.3f})"
)

# Keras MLP
fpr_keras, tpr_keras, _ = roc_curve(y_test, keras_probs)
roc_auc_keras = auc(fpr_keras, tpr_keras)
plt.plot(fpr_keras, tpr_keras, label=f"Keras MLP (AUC = {roc_auc_keras:.3f})")

# Ensemble
fpr_ensemble, tpr_ensemble, _ = roc_curve(y_test, ensemble_probs)
roc_auc_ensemble = auc(fpr_ensemble, tpr_ensemble)
plt.plot(fpr_ensemble, tpr_ensemble, label=f"Ensemble (AUC = {roc_auc_ensemble:.3f})")

# Plot diagonal line (random classifier)
plt.plot([0, 1], [0, 1], "k--")

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

## Using the API <a id="using-the-api"></a>

Now let's learn how to interact with the Heart Disease Prediction API.

In [ ]:
# Set API base URL - adjust this for your environment
API_BASE_URL = "http://localhost:8000"

# Check if API is running
try:
    response = requests.get(f"{API_BASE_URL}/health")
    if response.status_code == 200:
        print("API is running and healthy!")
        print(f"Response: {response.json()}")
    else:
        print(f"API returned status code {response.status_code}")
except requests.exceptions.ConnectionError:
    print("Could not connect to the API. Make sure it's running using:")
    print("  ./scripts/run_api.sh")
    print("\nFor this tutorial, we'll proceed with simulated API responses.")
    API_RUNNING = False
else:
    API_RUNNING = True

In [ ]:
# Get information about available models
if API_RUNNING:
    response = requests.get(f"{API_BASE_URL}/models/info")
    if response.status_code == 200:
        models_info = response.json()
        print("Available models:")
        print(json.dumps(models_info, indent=2))
    else:
        print(f"Error: {response.status_code} - {response.text}")
else:
    # Simulated response
    print("Simulated model information:")
    print(
        json.dumps(
            {
                "models_available": {"sklearn_mlp": True, "keras_mlp": True},
                "ensemble_available": True,
                "preprocessor_available": True,
            },
            indent=2,
        )
    )

In [ ]:
# Example patient data
sample_patient = {
    "age": 61,
    "sex": 1,
    "cp": 3,
    "trestbps": 140,
    "chol": 240,
    "fbs": 1,
    "restecg": 1,
    "thalach": 150,
    "exang": 1,
    "oldpeak": 2.4,
    "slope": 2,
    "ca": 1,
    "thal": 3,
}

print("Sample patient data:")
print(json.dumps(sample_patient, indent=2))

In [ ]:
# Make a prediction using the API
def make_prediction(patient_data, model=None):
    """Make a prediction using the Heart Disease Prediction API."""
    if API_RUNNING:
        url = f"{API_BASE_URL}/predict"
        if model:
            url += f"?model={model}"

        response = requests.post(url, json=patient_data)

        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None
    else:
        # Simulated response
        print("Simulating API prediction...")
        return {
            "prediction": 1,
            "probability": 0.9876,
            "risk_level": "HIGH",
            "interpretation": "HIGH RISK PREDICTION: 98.8% probability of heart disease\n\nKey risk factors identified:\n- Advanced age (over 55)\n- Male over 45\n- Chest pain at rest\n- Exercise-induced angina\n\nRecommendations:\n- Consult with a cardiologist\n- Consider stress test or other cardiac evaluations\n- Review medication and lifestyle modifications",
            "model_used": "ensemble",
        }


# Make prediction using ensemble (default)
prediction_result = make_prediction(sample_patient)
print("\nPrediction Result:")
print(json.dumps(prediction_result, indent=2))

In [ ]:
# Try predictions with different models
models = ["sklearn", "keras", "ensemble"]

results = {}
for model in models:
    result = make_prediction(sample_patient, model)
    if result:
        results[model] = result

# Compare results
for model, result in results.items():
    print(f"\n{model.upper()} Model:")
    print(f"  Prediction: {result['prediction']}")
    print(f"  Probability: {result['probability']:.4f}")
    print(f"  Risk Level: {result['risk_level']}")

## Batch Processing <a id="batch-processing"></a>

Now let's see how to perform batch predictions for multiple patients.

In [ ]:
# Generate a batch of sample patients
import random


def generate_sample_patients(count=10):
    """Generate a batch of random patient data for testing."""
    patients = []

    for _ in range(count):
        patient = {
            "age": random.randint(30, 80),
            "sex": random.randint(0, 1),
            "cp": random.randint(0, 3),
            "trestbps": random.randint(100, 180),
            "chol": random.randint(150, 350),
            "fbs": random.randint(0, 1),
            "restecg": random.randint(0, 2),
            "thalach": random.randint(100, 200),
            "exang": random.randint(0, 1),
            "oldpeak": round(random.uniform(0, 4), 1),
            "slope": random.randint(0, 2),
            "ca": random.randint(0, 3),
            "thal": random.randint(1, 3),
        }
        patients.append(patient)

    return patients


# Generate sample batch
sample_batch = generate_sample_patients(5)
print(f"Generated a batch of {len(sample_batch)} patients")
print("\nSample patient from batch:")
print(json.dumps(sample_batch[0], indent=2))

In [ ]:
# Make batch prediction
def make_batch_prediction(patients, model=None):
    """Make batch predictions using the Heart Disease Prediction API."""
    if API_RUNNING:
        url = f"{API_BASE_URL}/predict/batch"
        if model:
            url += f"?model={model}"

        response = requests.post(url, json=patients)

        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None
    else:
        # Simulated response
        print("Simulating API batch prediction...")
        predictions = []
        for _ in range(len(patients)):
            prob = random.uniform(0, 1)
            pred = 1 if prob >= 0.5 else 0
            risk = "HIGH" if prob >= 0.6 else ("MODERATE" if prob >= 0.3 else "LOW")
            predictions.append(
                {
                    "prediction": pred,
                    "probability": prob,
                    "risk_level": risk,
                    "model_used": "ensemble",
                }
            )

        return {
            "predictions": predictions,
            "performance_metrics": {
                "total_patients": len(patients),
                "processing_time_seconds": 0.125,
                "throughput_patients_per_second": len(patients) / 0.125,
                "num_chunks": 1,
                "chunk_size": 50,
                "num_workers": 4,
            },
        }


# Make batch prediction
batch_result = make_batch_prediction(sample_batch)

# Display results summary
if batch_result:
    print("\nBatch Prediction Results:")

    # Summarize predictions
    predictions = batch_result["predictions"]
    positive_count = sum(1 for p in predictions if p["prediction"] == 1)
    negative_count = len(predictions) - positive_count

    print(f"Total patients: {len(predictions)}")
    print(
        f"Positive predictions: {positive_count} ({positive_count/len(predictions):.1%})"
    )
    print(
        f"Negative predictions: {negative_count} ({negative_count/len(predictions):.1%})"
    )

    # Risk level breakdown
    risk_levels = {"LOW": 0, "MODERATE": 0, "HIGH": 0}
    for p in predictions:
        risk_levels[p["risk_level"]] += 1

    print("\nRisk level breakdown:")
    for level, count in risk_levels.items():
        if count > 0:
            print(f"  {level}: {count} ({count/len(predictions):.1%})")

    # Performance metrics
    if "performance_metrics" in batch_result:
        metrics = batch_result["performance_metrics"]
        print("\nPerformance metrics:")
        print(f"  Processing time: {metrics['processing_time_seconds']:.3f} seconds")
        print(
            f"  Throughput: {metrics['throughput_patients_per_second']:.2f} patients/second"
        )
        print(f"  Chunks: {metrics['num_chunks']} (size: {metrics['chunk_size']})")
        print(f"  Workers: {metrics['num_workers']}")

In [ ]:
# Check batch configuration
if API_RUNNING:
    response = requests.get(f"{API_BASE_URL}/batch/config")

    if response.status_code == 200:
        batch_config = response.json()
        print("Current Batch Configuration:")
        print(json.dumps(batch_config, indent=2))
    else:
        print(f"Error: {response.status_code} - {response.text}")
else:
    # Simulated response
    print("Simulated batch configuration:")
    print(
        json.dumps(
            {"batch_size": 50, "max_workers": 4, "performance_logging": True}, indent=2
        )
    )

In [ ]:
# Update batch configuration (if API is running)
if API_RUNNING:
    new_config = {"batch_size": 100, "max_workers": 8, "performance_logging": True}

    response = requests.post(f"{API_BASE_URL}/batch/config", json=new_config)

    if response.status_code == 200:
        updated_config = response.json()
        print("Updated Batch Configuration:")
        print(json.dumps(updated_config, indent=2))
    else:
        print(f"Error: {response.status_code} - {response.text}")

## Caching for Performance <a id="caching"></a>

Let's explore the caching system that improves prediction performance.

In [ ]:
# Check cache statistics
if API_RUNNING:
    response = requests.get(f"{API_BASE_URL}/cache/stats")

    if response.status_code == 200:
        cache_stats = response.json()
        print("Current Cache Statistics:")
        print(json.dumps(cache_stats, indent=2))
    else:
        print(f"Error: {response.status_code} - {response.text}")
else:
    # Simulated response
    print("Simulated cache statistics:")
    print(
        json.dumps(
            {
                "enabled": True,
                "max_size": 1000,
                "ttl_seconds": 3600,
                "entries": 25,
                "hits": 120,
                "misses": 30,
                "hit_rate": 0.8,
                "evictions": 0,
                "created_at": "2025-03-31T15:00:00.000000",
            },
            indent=2,
        )
    )

In [ ]:
# Demonstrate cache performance improvement
if API_RUNNING:
    # Make first prediction (cache miss)
    print("Making first prediction (should be a cache miss)...")
    start_time = time.time()
    result1 = make_prediction(sample_patient)
    first_time = time.time() - start_time
    print(f"Time taken: {first_time:.4f} seconds")

    # Make second prediction (cache hit)
    print("\nMaking second prediction with same data (should be a cache hit)...")
    start_time = time.time()
    result2 = make_prediction(sample_patient)
    second_time = time.time() - start_time
    print(f"Time taken: {second_time:.4f} seconds")

    # Calculate speedup
    if first_time > 0 and second_time > 0:
        speedup = first_time / second_time
        print(f"\nCache speedup: {speedup:.2f}x faster")

    # Check updated cache statistics
    response = requests.get(f"{API_BASE_URL}/cache/stats")
    if response.status_code == 200:
        cache_stats = response.json()
        print("\nUpdated Cache Statistics:")
        print(json.dumps(cache_stats, indent=2))
else:
    # Simulated cache performance
    print("Simulating cache performance...")
    print("First prediction (cache miss): 0.1500 seconds")
    print("Second prediction (cache hit): 0.0100 seconds")
    print("Cache speedup: 15.00x faster")

In [ ]:
# Update cache configuration (if API is running)
if API_RUNNING:
    new_cache_config = {
        "enabled": True,
        "max_size": 2000,
        "ttl": 7200,  # 2 hours in seconds
    }

    response = requests.post(f"{API_BASE_URL}/cache/config", json=new_cache_config)

    if response.status_code == 200:
        updated_config = response.json()
        print("Updated Cache Configuration:")
        print(json.dumps(updated_config, indent=2))
    else:
        print(f"Error: {response.status_code} - {response.text}")

In [ ]:
# Demonstrate batch processing with cache
# First clear the cache to start fresh
if API_RUNNING:
    response = requests.post(f"{API_BASE_URL}/cache/clear")
    if response.status_code == 200:
        print("Cache cleared successfully!")

    # Generate a larger batch for testing
    large_batch = generate_sample_patients(50)

    # First batch prediction (cache misses)
    print("\nMaking first batch prediction (should be cache misses)...")
    start_time = time.time()
    first_batch_result = make_batch_prediction(large_batch)
    first_batch_time = time.time() - start_time

    if "performance_metrics" in first_batch_result:
        metrics1 = first_batch_result["performance_metrics"]
        print(f"Time taken: {metrics1['processing_time_seconds']:.4f} seconds")
        print(
            f"Throughput: {metrics1['throughput_patients_per_second']:.2f} patients/second"
        )
    else:
        print(f"Time taken: {first_batch_time:.4f} seconds")

    # Second batch prediction with same data (cache hits)
    print("\nMaking second batch prediction with same data (should be cache hits)...")
    start_time = time.time()
    second_batch_result = make_batch_prediction(large_batch)
    second_batch_time = time.time() - start_time

    if "performance_metrics" in second_batch_result:
        metrics2 = second_batch_result["performance_metrics"]
        print(f"Time taken: {metrics2['processing_time_seconds']:.4f} seconds")
        print(
            f"Throughput: {metrics2['throughput_patients_per_second']:.2f} patients/second"
        )

        # Calculate speedup
        if (
            metrics1["processing_time_seconds"] > 0
            and metrics2["processing_time_seconds"] > 0
        ):
            speedup = (
                metrics1["processing_time_seconds"]
                / metrics2["processing_time_seconds"]
            )
            throughput_increase = (
                metrics2["throughput_patients_per_second"]
                / metrics1["throughput_patients_per_second"]
            )
            print(f"\nCache speedup: {speedup:.2f}x faster")
            print(f"Throughput increase: {throughput_increase:.2f}x")
    else:
        print(f"Time taken: {second_batch_time:.4f} seconds")
        if first_batch_time > 0 and second_batch_time > 0:
            speedup = first_batch_time / second_batch_time
            print(f"\nCache speedup: {speedup:.2f}x faster")

    # Check cache statistics after batch operations
    response = requests.get(f"{API_BASE_URL}/cache/stats")
    if response.status_code == 200:
        cache_stats = response.json()
        print("\nCache Statistics After Batch Operations:")
        print(json.dumps(cache_stats, indent=2))
else:
    # Simulated batch cache performance
    print("Simulating batch processing with caching...")
    print(
        "First batch (50 patients, cache misses): 0.5000 seconds (100.00 patients/second)"
    )
    print(
        "Second batch (50 patients, cache hits): 0.0050 seconds (10000.00 patients/second)"
    )
    print("Cache speedup: 100.00x faster")

## System Architecture <a id="system-architecture"></a>

Let's examine the system architecture of the Heart Disease Prediction system.

In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import numpy as np

# Create a simplified architecture diagram
fig, ax = plt.subplots(figsize=(14, 10))

# Define component colors
data_color = "#8dd3c7"
model_color = "#fb8072"
api_color = "#80b1d3"
client_color = "#bebada"
utils_color = "#fdb462"

# Define arrow properties
arrow_props = dict(
    arrowstyle="->", connectionstyle="arc3,rad=0.1", color="gray", linewidth=1.5
)

# Create background boxes
# Data Layer
data_layer = patches.Rectangle(
    (0.1, 0.7),
    0.35,
    0.25,
    fill=True,
    alpha=0.3,
    color=data_color,
    transform=ax.transAxes,
)
ax.add_patch(data_layer)
ax.text(0.275, 0.9, "Data Layer", ha="center", fontsize=14, transform=ax.transAxes)

# Model Layer
model_layer = patches.Rectangle(
    (0.1, 0.4),
    0.35,
    0.25,
    fill=True,
    alpha=0.3,
    color=model_color,
    transform=ax.transAxes,
)
ax.add_patch(model_layer)
ax.text(0.275, 0.6, "Model Layer", ha="center", fontsize=14, transform=ax.transAxes)

# API Layer
api_layer = patches.Rectangle(
    (0.5, 0.4),
    0.35,
    0.55,
    fill=True,
    alpha=0.3,
    color=api_color,
    transform=ax.transAxes,
)
ax.add_patch(api_layer)
ax.text(0.675, 0.9, "API Layer", ha="center", fontsize=14, transform=ax.transAxes)

# Client Layer
client_layer = patches.Rectangle(
    (0.5, 0.1),
    0.35,
    0.25,
    fill=True,
    alpha=0.3,
    color=client_color,
    transform=ax.transAxes,
)
ax.add_patch(client_layer)
ax.text(0.675, 0.3, "Client Layer", ha="center", fontsize=14, transform=ax.transAxes)

# Utils (Shared)
utils_layer = patches.Rectangle(
    (0.1, 0.1),
    0.35,
    0.25,
    fill=True,
    alpha=0.3,
    color=utils_color,
    transform=ax.transAxes,
)
ax.add_patch(utils_layer)
ax.text(0.275, 0.3, "Utilities", ha="center", fontsize=14, transform=ax.transAxes)

# Add components
# Data Layer Components
ax.text(0.15, 0.85, "- raw_data", fontsize=10, transform=ax.transAxes)
ax.text(0.15, 0.82, "- preprocessor", fontsize=10, transform=ax.transAxes)
ax.text(0.15, 0.79, "- train/val/test splits", fontsize=10, transform=ax.transAxes)
ax.text(0.15, 0.76, "- feature engineering", fontsize=10, transform=ax.transAxes)

# Model Layer Components
ax.text(0.15, 0.55, "- scikit-learn MLP", fontsize=10, transform=ax.transAxes)
ax.text(0.15, 0.52, "- Keras MLP", fontsize=10, transform=ax.transAxes)
ax.text(0.15, 0.49, "- Ensemble combiner", fontsize=10, transform=ax.transAxes)
ax.text(0.15, 0.46, "- Prediction cache", fontsize=10, transform=ax.transAxes)
ax.text(0.15, 0.43, "- Evaluation metrics", fontsize=10, transform=ax.transAxes)

# API Layer Components
ax.text(0.55, 0.85, "- FastAPI app", fontsize=10, transform=ax.transAxes)
ax.text(0.55, 0.82, "- Endpoints:", fontsize=10, transform=ax.transAxes)
ax.text(0.57, 0.79, "* /predict", fontsize=10, transform=ax.transAxes)
ax.text(0.57, 0.76, "* /predict/batch", fontsize=10, transform=ax.transAxes)
ax.text(0.57, 0.73, "* /batch/config", fontsize=10, transform=ax.transAxes)
ax.text(0.57, 0.70, "* /cache/stats", fontsize=10, transform=ax.transAxes)
ax.text(0.57, 0.67, "* /cache/config", fontsize=10, transform=ax.transAxes)
ax.text(0.57, 0.64, "* /cache/clear", fontsize=10, transform=ax.transAxes)
ax.text(0.57, 0.61, "* /models/info", fontsize=10, transform=ax.transAxes)
ax.text(0.57, 0.58, "* /health", fontsize=10, transform=ax.transAxes)
ax.text(0.55, 0.55, "- Request validation", fontsize=10, transform=ax.transAxes)
ax.text(0.55, 0.52, "- Error handling", fontsize=10, transform=ax.transAxes)
ax.text(0.55, 0.49, "- Parallel processing", fontsize=10, transform=ax.transAxes)
ax.text(0.55, 0.46, "- Thread pooling", fontsize=10, transform=ax.transAxes)
ax.text(0.55, 0.43, "- Performance logging", fontsize=10, transform=ax.transAxes)

# Client Layer Components
ax.text(0.55, 0.25, "- Web UI", fontsize=10, transform=ax.transAxes)
ax.text(0.55, 0.22, "- CLI client", fontsize=10, transform=ax.transAxes)
ax.text(0.55, 0.19, "- Batch client", fontsize=10, transform=ax.transAxes)
ax.text(0.55, 0.16, "- Integration examples", fontsize=10, transform=ax.transAxes)
ax.text(0.55, 0.13, "- Test scripts", fontsize=10, transform=ax.transAxes)

# Utils Components
ax.text(0.15, 0.25, "- Configuration loader", fontsize=10, transform=ax.transAxes)
ax.text(0.15, 0.22, "- Logging setup", fontsize=10, transform=ax.transAxes)
ax.text(0.15, 0.19, "- Path management", fontsize=10, transform=ax.transAxes)
ax.text(0.15, 0.16, "- Error handling", fontsize=10, transform=ax.transAxes)
ax.text(0.15, 0.13, "- Visualization helpers", fontsize=10, transform=ax.transAxes)

# Add arrows for data flow
# Data Layer -> Model Layer
ax.annotate(
    "",
    xy=(0.275, 0.65),
    xytext=(0.275, 0.7),
    arrowprops=arrow_props,
    transform=ax.transAxes,
)

# Model Layer -> API Layer
ax.annotate(
    "",
    xy=(0.5, 0.6),
    xytext=(0.45, 0.525),
    arrowprops=arrow_props,
    transform=ax.transAxes,
)

# API Layer -> Client Layer
ax.annotate(
    "",
    xy=(0.675, 0.35),
    xytext=(0.675, 0.4),
    arrowprops=arrow_props,
    transform=ax.transAxes,
)

# Utils -> All Layers
ax.annotate(
    "",
    xy=(0.275, 0.4),
    xytext=(0.275, 0.35),
    arrowprops=arrow_props,
    transform=ax.transAxes,
)
ax.annotate(
    "",
    xy=(0.5, 0.25),
    xytext=(0.45, 0.25),
    arrowprops=arrow_props,
    transform=ax.transAxes,
)

# Remove axes
ax.set_axis_off()

# Add title
plt.suptitle("Heart Disease Prediction System Architecture", fontsize=18, y=0.98)

# Add legend for color coding
legend_elements = [
    patches.Patch(facecolor=data_color, alpha=0.3, label="Data Processing"),
    patches.Patch(facecolor=model_color, alpha=0.3, label="Model Components"),
    patches.Patch(facecolor=api_color, alpha=0.3, label="API Services"),
    patches.Patch(facecolor=client_color, alpha=0.3, label="Client Applications"),
    patches.Patch(facecolor=utils_color, alpha=0.3, label="Utility Functions"),
]
ax.legend(
    handles=legend_elements,
    loc="lower center",
    bbox_to_anchor=(0.5, 0.02),
    ncol=5,
    fontsize=10,
)

plt.tight_layout()
plt.show()

### Architecture Highlights

The Heart Disease Prediction system employs a layered architecture that separates concerns and promotes maintainability:

1. **Data Layer**:
   - Responsible for data acquisition, preprocessing, and feature engineering
   - Manages train/validation/test splits and data transformations
   - Stores processed data and preprocessing artifacts

2. **Model Layer**:
   - Implements multiple machine learning models (scikit-learn MLP, Keras MLP)
   - Provides ensemble prediction capability
   - Includes model evaluation metrics and interpretation
   - Implements prediction caching for improved performance

3. **API Layer**:
   - Exposes models through REST endpoints using FastAPI
   - Provides single and batch prediction capabilities
   - Implements parallel processing for batch predictions
   - Includes configuration endpoints for caching and batch settings
   - Implements comprehensive error handling and validation

4. **Client Layer**:
   - Multiple interfaces for interacting with the system (Web UI, CLI, etc.)
   - Provides examples for integration with other systems
   - Includes testing scripts for validation

5. **Utilities (Shared)**:
   - Common functionality used across layers
   - Configuration management
   - Logging and path handling
   - Visualization helpers

### Key Performance Features

The system implements several optimizations for high performance:

1. **Prediction Caching**:
   - LRU (Least Recently Used) caching for repeated predictions
   - Configurable TTL (Time-To-Live) for cache entries
   - Hash-based cache keys for efficient lookups
   - Statistics tracking for monitoring cache performance

2. **Parallel Processing**:
   - Chunking of large batches for efficient processing
   - Thread pool for parallel execution of prediction chunks
   - Configurable chunk size and worker count for tuning

3. **Model Selection**:
   - Multiple model options with different performance characteristics
   - Ensemble approach for improved accuracy
   - Fallback mechanisms for handling model unavailability

This architecture ensures the system is scalable, maintainable, and performs well under various workloads.

## Next Steps <a id="next-steps"></a>

Based on the project roadmap, here are the next steps for enhancing the Heart Disease Prediction system:

### Security Enhancements

1. **API Authentication**:
   - Implement API key authentication
   - Add user management capabilities
   - Implement role-based access control

2. **Deployment Improvements**:
   - Implement backup and recovery procedures
   - Create environment-specific configuration for dev/staging/prod
   - Enhance Docker deployment with multi-stage builds

### Performance Optimizations

1. **Distributed Caching**:
   - Implement Redis-based caching for multi-instance deployments
   - Add cache persistence for resilience

2. **Model Optimization**:
   - Implement model quantization for faster inference
   - Explore TensorFlow Lite for edge deployment

### Feature Enhancements

1. **Advanced Models**:
   - Add gradient boosting model (XGBoost, LightGBM)
   - Implement Bayesian neural networks for uncertainty quantification
   - Add explainability with SHAP/LIME integration

2. **User Experience**:
   - Create downloadable report format for predictions
   - Enhance visualization components
   - Implement batch upload interface

### Monitoring and Telemetry

1. **Performance Monitoring**:
   - Add dashboard for monitoring cache performance and API throughput
   - Implement telemetry for tracking prediction errors
   - Create centralized logging system

2. **Model Drift Detection**:
   - Implement automated monitoring for model drift
   - Create alerts for performance degradation
   - Add scheduled retraining capability

By implementing these enhancements, the Heart Disease Prediction system will become even more robust, secure, and user-friendly.

## Conclusion

This tutorial has provided a comprehensive overview of the Heart Disease Prediction system, including:

- Data exploration and feature engineering
- Model training and evaluation
- Using the API for predictions
- Implementing batch processing for efficiency
- Leveraging caching for improved performance
- Understanding the system architecture

The system demonstrates good practices in machine learning engineering, with a focus on:

- Model performance and evaluation
- Scalable and efficient API design
- Performance optimization through caching and parallel processing
- Comprehensive error handling and validation
- Clear system architecture with separation of concerns

By following this tutorial, you should now have a good understanding of how to use and extend the Heart Disease Prediction system for your own applications.